In [1]:
import util
from journal_api.journal_api import JournalAPI
from IPython.display import JSON
import xml.etree.ElementTree as ET
import json
import numpy as np
import pandas as pd
import logging
import yaml
from journal_api import semanticscholar_api

2023-08-25 17:02:59,728 - INFO - NumExpr defaulting to 8 threads.
2023-08-25 17:03:05,570 - DEBUG - Falling back to TensorFlow client; we recommended you install the Cloud TPU client directly with pip install cloud-tpu-client.
2023-08-25 17:03:05,831 - DEBUG - Creating converter from 7 to 5
2023-08-25 17:03:05,832 - DEBUG - Creating converter from 5 to 7
2023-08-25 17:03:05,832 - DEBUG - Creating converter from 7 to 5
2023-08-25 17:03:05,832 - DEBUG - Creating converter from 5 to 7


In [2]:
with open('logging_config.yaml', 'r') as config_file:
    log_config = yaml.safe_load(config_file)
logging.config.dictConfig(log_config)

In [3]:
title = "Design and Control of Roller Grasper V2 for In-Hand Manipulation"

In [4]:
journal = JournalAPI()
dt = journal.search(title, 2018, find_references=False, fix_references=False)
JSON(dt)

2023-08-25 17:03:11,392 - DEBUG - ss.search: Found! title=Design and Control of Roller Grasper V2 for In-Hand Manipulation


<IPython.core.display.JSON object>

In [5]:
def source_target(source, target):
    ret = pd.DataFrame(columns=["source", "target"])
    for t in target:
        ret = ret.append({"source": source, "target": t}, ignore_index=True)
    return ret

In [6]:
def source_target_generator(title, title_list, level = 1, start_year = 2017, fix_references = False):
    ret = pd.DataFrame(columns=["source", "target"])
    if(level > 0):
        logging.info(f"source_target_generator: level={level}, len={len(title_list)}, title={title}")
        for index, title in enumerate(title_list):
            logging.info(f"source_target_generator: ({index+1}/{len(title_list)}) title={title}")
            src = journal.search(title, start_year, find_references=True, fix_references=fix_references)
            if src != None:
                # TODO: Create global variable to store detailed info of paper
                st = source_target(src["title"],src["references"])
                ret = pd.concat([ret, st], ignore_index=True)
                lv = level - 1
                if lv == 1:
                    ret = pd.concat([ret, source_target_generator(src["title"], src["references"], lv, start_year)], ignore_index=True)
                else:
                    ret = pd.concat([ret, source_target_generator(src["title"], src["references"], lv, start_year, fix_references)], ignore_index=True)
            
    return ret

In [7]:
x = source_target_generator("", [title], 3, 2018, True)

2023-08-25 17:03:16,993 - INFO - source_target_generator: level=3, len=1, title=
2023-08-25 17:03:16,994 - INFO - source_target_generator: (1/1) title=Design and Control of Roller Grasper V2 for In-Hand Manipulation
2023-08-25 17:03:17,513 - DEBUG - ss.search: Found! title=Design and Control of Roller Grasper V2 for In-Hand Manipulation
2023-08-25 17:03:18,191 - DEBUG - ss.search: Found! title=Concept2Robot: Learning manipulation concepts from instructions and human demonstrations
2023-08-25 17:03:18,845 - DEBUG - ss.search: Found! title=Design of a Roller-Based Dexterous Hand for Object Grasping and Within-Hand Manipulation
2023-08-25 17:03:19,485 - DEBUG - ss.search: Found! title=Design
and Analysis of a Multimodal Grasper Having Shape Conformity and Within-Hand Manipulation With
Adjustable Contact Forces
2023-08-25 17:03:19,982 - DEBUG - ss.search: Found! title=A Century of Robotic Hands
2023-08-25 17:03:19,983 - WARNING - ss._remake_tldr: No tldr!
2023-08-25 17:03:19,984 - WARNING 

In [8]:
x

,source,target
0,Design and Control of Roller Grasper V2 for In...,Concept2Robot: Learning manipulation concepts ...
1,Design and Control of Roller Grasper V2 for In...,Design of a Roller-Based Dexterous Hand for Ob...
2,Design and Control of Roller Grasper V2 for In...,Design\nand Analysis of a Multimodal Grasper H...
3,Design and Control of Roller Grasper V2 for In...,A Century of Robotic Hands
4,Design and Control of Roller Grasper V2 for In...,Dual-Arm In-Hand Manipulation and Regrasping U...
...,...,...
982,Distributed Distributional Deterministic Polic...,"r “ ψ p cos ´ 1 p obj yaxis || xy , obj target..."
983,On Policy Learning Robust to Irreversible Even...,Enhance In-Hand Dexterous Micromanipulation by...
984,In-Hand Manipulation via Motion Cones,"Pushing revisited: Differential flatness, traj..."
985,"Pushing revisited: Differential flatness, traj...",A convex polynomial model for planar sliding m...


In [9]:
x.to_csv("test_1.csv", index=False)

In [ ]:
from collections import defaultdict
from pyvis.network import Network

net = Network("1200px", "1200px", notebook=True)

# Example dummy data
sources = x["source"]
targets = x["target"]
sources_short = [source[:100] + "..." for source in sources]
targets_short = [target[:100] + "..." for target in targets]

# Create a dictionary to store the count of each target
target_counts = defaultdict(int)
for t in targets_short:
    target_counts[t] += 10
    if t == "...":
        target_counts[t] = 1

# Create unique node labels for each occurrence of a node
unique_nodes = {}
for node in set(sources_short + targets_short):
    count = target_counts[node]
    size = count if count > 1 else 1  # Ensuring the smallest size is 1
    unique_label = f"{node} ({count})"
    net.add_node(unique_label, size=size)
    unique_nodes[node] = unique_label

# Add edges to the graph using the unique node labels
for s, t in zip(sources_short, targets_short):
    net.add_edge(unique_nodes[s], unique_nodes[t])

# Step 3: Visualize the graph in Jupyter Notebook
net.show("network_graph2.html")


In [ ]:
ref_pd = util.string_to_df(xy["refs"][1])

In [ ]:
xy_refs = generate_title_auths_refs(ref_pd)
xy_refs

In [ ]:
xy_refs["title"]

In [ ]:
json_data = xy_refs["title"].to_json(orient='records')

# Specify the output file path
output_file = 'data.json'

# Write JSON data to a file
with open(output_file, 'w') as file:
    file.write(json_data)

In [ ]:
crossref = util.crossref_search(title)
json.dumps(crossref, ensure_ascii=False)

In [ ]:
cref = crossref_search_by_doi("10.1177/0278364914559753")

In [ ]:
crossref = crossref_similar_search(title,10)
crossref

In [ ]:
st = "ALVINN: An Autonomous Land Vehicle in a Neural Network"
tt = google_search_books(st)
tt

In [ ]:
arxiv_search("A reduction of imitation learning and structured prediction to no-regret online learning")

In [ ]:
import requests
import json

def crossref_keywords_search(keywords, rows):
    """
    crossref_search
    """
    # Specify the API endpoint
    ENDPOINT = "https://api.crossref.org/works"

    # Set up the query parameters
    params = {
        "query": " OR ".join(keywords),
        "rows": rows  # Number of results to retrieve
    }

    # Send the API request
    response = requests.get(ENDPOINT, params=params, timeout=10)

    # Parse the JSON response
    data = response.json()

    paper_list = []
    # Extract the relevant information from the response
    if data["status"] == "ok":
        papers = data["message"]["items"]
        if len(papers) > 0:
            for paper in papers:
                paper_list.append(paper)
    return paper_list

keywords = ["machine learning", "deep learning", "artificial intelligence"]
dois = crossref_keywords_search(keywords, 10)
print(json.dumps(dois))

In [ ]:
" OR ".join(keywords)

In [ ]:
import requests

def search_similar_papers(keywords):
    base_url = 'https://api.crossref.org/works'
    
    # Create the query string with the specified keywords
    query = ' '.join(f'title:{keyword} abstract:{keyword}' for keyword in keywords)
    
    # Prepare the request URL
    url = f'{base_url}?query={query}&rows=10'  # Limiting the response to 10 papers
    
    try:
        # Send the HTTP GET request
        response = requests.get(url)
        
        # Check if the request was successful (status code 200)
        if response.status_code == 200:
            data = response.json()
            
            # Extract the relevant information from the response
            papers = data['message']['items']
            
            # Process the papers
            for paper in papers:
                title = paper['title'][0]
                authors = ""
                if paper.get("author") != None:
                    lst_authors = paper.get("author")
                    for author in lst_authors:
                        auth_name = author["given"] + " " + author["family"]
                        authors = authors + auth_name + ", "
                
                doi = paper['DOI']
                
                # Print the information
                print(f'Title: {title}')
                print(f'Authors: {authors}')
                print(f'DOI: {doi}')
                print('---')
        else:
            print(f'Request failed with status code {response.status_code}')
    except requests.exceptions.RequestException as e:
        print(f'An error occurred: {e}')

# Example usage
keywords = ['finger joint montions', 'spherical rolling fingertips', 'handcrafted control', 'sophisticated object manipulation', 'roller grasper']
search_similar_papers(keywords)

In [ ]:
t = [True, True, True, True]
t
if not any(t):
    print("True")

In [ ]:
for i in range(3):
    print(i)

In [ ]:
from pyvis.network import Network
import networkx as nx

# Create a sample network graph using NetworkX
G = nx.Graph()
G.add_edges_from([(1, 2), (2, 3), (3, 4), (4, 1)])

# Create a PyVis Network object
net = Network(notebook=True)

# Add nodes and edges to the PyVis Network
for node in G.nodes():
    net.add_node(node, label=f'Node {node}')

for edge in G.edges():
    net.add_edge(edge[0], edge[1])

# Define the click handler function
def show_text(node_id):
    node = net.nodes[node_id]
    if 'title' in node:
        del node['title']
    else:
        node['title'] = f'This is Node {node_id}'

# Bind the click handler to the 'on_click' event
net.add_event_listener("selectNode", show_text)

# Show the network graph
net.show("example.html")